### 0.Import libraries

In [81]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import re


### 1.Load Dataset

In [34]:
df_origin_test= pd.read_csv('data/test_set.csv')
df_origin_train = pd.read_csv('data/train_set.csv')

df_train = df_origin_train.copy()
df_test = df_origin_test.copy()

### 2.Data analysis

In [35]:
df_train.head(3)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,84094,23943,5,Leonoray,18.0,84355102.0,Journalist,70260.20,5564.016667,9.0,...,Bad,2228.79,25.308445,217.0,Yes,246.378645,99.939577,Low_spent_Small_value_payments,78.800397,Poor
1,46702,29066,5,Osamua,21.0,981149909.0,Teacher,18001.59,1258.132500,7.0,...,Bad,2225.58,32.088726,240.0,NM,24.986447,45.523998,Low_spent_Small_value_payments,343.581411,Poor
2,147514,8183,5,Benf,47.0,324295086.0,Developer,9824.31,707.692500,7.0,...,Standard,1233.96,25.500503,227.0,No,0.000000,22.006889,Low_spent_Medium_value_payments,322.170689,Good


In [36]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        70000 non-null  int64  
 1   Customer_ID               70000 non-null  int64  
 2   Month                     70000 non-null  int64  
 3   Name                      70000 non-null  object 
 4   Age                       70000 non-null  float64
 5   SSN                       70000 non-null  float64
 6   Occupation                70000 non-null  object 
 7   Annual_Income             70000 non-null  float64
 8   Monthly_Inhand_Salary     70000 non-null  float64
 9   Num_Bank_Accounts         70000 non-null  float64
 10  Num_Credit_Card           70000 non-null  float64
 11  Interest_Rate             70000 non-null  float64
 12  Num_of_Loan               70000 non-null  float64
 13  Type_of_Loan              70000 non-null  object 
 14  Delay_

In [37]:
df_train.shape, df_test.shape

((70000, 28), (30000, 27))

**Credit Score is the output y**

In [50]:
#好干净的数据
df_train['Credit_Score'].isna().sum()

0

检查列名的一致性

In [45]:
#一致性通过
(df_train.columns[:-1]!=df_test.columns).sum()

In [46]:
df = pd.concat([df_train, df_test], ignore_index=True)
df.shape

(100000, 28)

In [47]:
df['Credit_Score'].isna().sum()

30000

In [52]:
#计算合并后的df缺失值的数量（结果符合预期）
df.isna().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                            0
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary           0
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                    0
Delay_from_due_date             0
Num_of_Delayed_Payment          0
Changed_Credit_Limit            0
Num_Credit_Inquiries            0
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age              0
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly         0
Payment_Behaviour               0
Monthly_Balance                 0
Credit_Score                30000
dtype: int64

查看 df中每个对象类型列的唯一值分布情况

In [54]:
for col in object_col:
    print('Column Name: '+col)
    print("**"*35)
    print(df_train[col].value_counts(dropna=False))
    print('END', "--"*35, '\n')

Column Name: Name
**********************************************************************
Name
Stevex              36
Strupczewskid       34
Raymondr            34
Vaughanl            33
Andreaj             33
                    ..
Armstrongk           1
Flaki                1
Niveditak            1
Alexe                1
Oleg Vukmanovicb     1
Name: count, Length: 10127, dtype: int64
END ---------------------------------------------------------------------- 

Column Name: Occupation
**********************************************************************
Occupation
Lawyer           4887
Architect        4807
Engineer         4790
Mechanic         4772
Developer        4767
Accountant       4742
Scientist        4704
Media_Manager    4692
Teacher          4622
Entrepreneur     4618
Journalist       4588
Doctor           4557
Manager          4501
Musician         4482
Writer           4471
Name: count, dtype: int64
END ---------------------------------------------------------------------

### 3.Data Preprocessing

In [106]:
#非数值列名
object_col = df.describe(include='O').columns
object_col

Index(['Occupation', 'Type_of_Loan', 'Credit_Mix', 'Payment_of_Min_Amount',
       'Payment_Behaviour'],
      dtype='object')

In [ ]:
del df['ID'] # Identification 
del df['Name'] # Name of client 
del df['SSN'] # SSN (social security number of a person)

Type of loans

In [84]:
df['Type_of_Loan'].value_counts(dropna=False).head(20)

Type_of_Loan
no data                                11408
not specified                           1408
credit-builder loan                     1280
personal loan                           1272
debt consolidation loan                 1264
student loan                            1240
payday loan                             1200
mortgage loan                           1176
auto loan                               1152
home equity loan                        1136
personal loan,student loan               320
not specified,payday loan                272
mortgage loan,home equity loan           264
student loan,payday loan                 256
credit-builder loan,not specified        248
student loan,credit-builder loan         248
payday loan,auto loan                    240
payday loan,debt consolidation loan      240
payday loan,personal loan                232
auto loan,personal loan                  232
Name: count, dtype: int64

In [85]:
#数据清洗，使得贷款类型的格式标准化。这个函数将字符串转换为小写，去除 "and " 字符串和逗号后的空格，并删除字符串两端的空格
df['Type_of_Loan'] = df['Type_of_Loan'].apply(lambda x: x.lower().replace('and ', '').replace(', ', ',').strip() if pd.notna(x) else x)

In [ ]:
df.groupby('Customer_ID')['Type_of_Loan'].value_counts(dropna=False)

In [ ]:
df.groupby('Customer_ID')['Type_of_Loan'].apply(list)

In [88]:
#获取 Type_of_Loan 列中所有不同的贷款类型，并且为每个不同的贷款类型赋予一个唯一的整数标识。
def get_Diff_Values_Colum(df_column, diff_value=[], sep=',', replace=''):   
    column = df_column.dropna()
    for i in column:
        if sep not in i and i not in diff_value:
            diff_value.append(i)
        else:
            for data in map(lambda x:x.strip(), re.sub(replace, '', i).split(sep)):
                if not data in diff_value:
                    diff_value.append(data)
    return dict(enumerate(sorted(diff_value)))

get_Diff_Values_Colum(df['Type_of_Loan'])

{0: 'auto loan',
 1: 'credit-builder loan',
 2: 'debt consolidation loan',
 3: 'home equity loan',
 4: 'mortgage loan',
 5: 'no data',
 6: 'not specified',
 7: 'payday loan',
 8: 'personal loan',
 9: 'student loan'}

Occupation

In [95]:
df['Occupation'].value_counts()

Occupation
Lawyer           7096
Engineer         6864
Architect        6824
Mechanic         6776
Scientist        6744
Accountant       6744
Developer        6720
Media_Manager    6720
Teacher          6672
Entrepreneur     6648
Doctor           6568
Journalist       6536
Manager          6432
Musician         6352
Writer           6304
Name: count, dtype: int64

In [96]:
df['Credit_Mix'].value_counts()

Credit_Mix
Standard    45848
Good        30384
Bad         23768
Name: count, dtype: int64

In [ ]:
df.select_dtypes(include="object")
#[ 'Occupation', 'Type_of_Loan', 'Credit_Mix',
#'Payment_of_Min_Amount', 'Payment_Behaviour', 'Credit_Score～']

In [98]:
df['Credit_Score'].value_counts()

Credit_Score
Standard    37080
Poor        20399
Good        12521
Name: count, dtype: int64

In [99]:
m = {"Poor":0, "Standard":1, "Good":2}

df['Credit_Score'] = df['Credit_Score'].map(m)

In [102]:
y = df['Credit_Score']
X = df.drop(columns=['Credit_Score'], axis=1)

In [103]:
categorical_cols = [col for col in X.columns if X[col].dtype == 'object']
categorical_cols

['Occupation',
 'Type_of_Loan',
 'Credit_Mix',
 'Payment_of_Min_Amount',
 'Payment_Behaviour']

In [104]:
low_cardinality_categorical_cols = [col for col in X.columns if 
                                    (X[col].dtype == 'object') & 
                                    (X[col].nunique()<4)]
low_cardinality_categorical_cols

['Credit_Mix', 'Payment_of_Min_Amount']

In [105]:
high_cardinality_categorical_cols = [col for col in X.columns if 
                                    (X[col].dtype == 'object') & 
                                    (X[col].nunique()>3)]

high_cardinality_categorical_cols

['Occupation', 'Type_of_Loan', 'Payment_Behaviour']